In [ ]:
from multiprocessing import Pool
import spacy
from spacy.matcher import Matcher
from tqdm import tqdm
import nltk
import json
import string
import multiprocessing

In [ ]:
blacklist = set(["-PRON-", "actually", "likely", "possibly", "want",
                 "make", "my", "someone", "sometimes_people", "sometimes", "would", "want_to",
                 "one", "something", "sometimes", "everybody", "somebody", "could", "could_be"
                 ])
nltk.download('stopwords', quiet=True)
nltk_stopwords = nltk.corpus.stopwords.words('english')
CPNET_VOCAB = None
PATTERN_PATH = None
nlp = None
matcher = None

In [ ]:
def load_cpnet_vocab(cpnet_vocab_path):
    '''
    This function load the vocab path
    '''
    with open(cpnet_vocab_path, "r", encoding="utf8") as fin:
        cpnet_vocab = [l.strip() for l in fin]
    cpnet_vocab = [c.replace("_", " ") for c in cpnet_vocab]
    return cpnet_vocab
vocab = load_cpnet_vocab("../input/cp-net-en/CPnet_en/concept.txt")

In [ ]:
def create_pattern(nlp, doc, debug = False):
    pronoun_list = set(["my", "you", "it", "its", "your", "i", "he", "she", "his", "her", "they", "them", "their", "our", "we"])
    if len(doc) >=5 or doc[0].text in pronoun_list or doc[-1].text in pronoun_list or \
        all([(token.text in nltk_stopwords or token.lemma_ in nltk_stopwords or token.lemma_ in blacklist) for token in doc]):
        # ignore those concepts become pattern
        return None
    pattern = []
    for token in doc:
        # lemmatize the type
        pattern.append({"LEMMA":token.lemma_})
    return pattern

def create_matcher_pattern(cpnet_vocab_path, output_path, debug = False):
    '''
    This function create the matcher patterns
    '''
    cpnet_vocab = load_cpnet_vocab(cpnet_vocab_path)
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])
    docs = nlp.pipe(cpnet_vocab)
    all_patterns = {}
    for doc in tqdm(docs, total = len(cpnet_vocab)):
        pattern = create_pattern(nlp,doc,debug)
        if pattern is None:
            continue
        # create mapping
        all_patterns["_".join(doc.text.split(" "))] = pattern
    with open(output_path, "w", encoding="utf8") as fout:
        json.dump(all_patterns, fout)

In [ ]:
# cores = multiprocessing.cpu_count()
# pool = multiprocessing.Pool(processes=cores)
# pool.map(create_matcher_pattern,kwargs = {"cpnet_vocab_path":"../input/cp-net-en/CPnet_en/concept.txt", "output_path":"./"})
create_matcher_pattern("../input/cp-net-en/CPnet_en/concept.txt", "file.json")